In [3]:
import json
import os

In [4]:
def import_metrics_json(models_path, run_num):
    """
    Looks in {models_path}/{run_num}/metrics.json and returns the contents as a
    Python dictionary. Returns None if the path does not exist.
    """
    path = os.path.join(models_path, str(run_num), "metrics.json")
    if not os.path.exists(path):
        return None
    with open(path, "r") as f:
        return json.load(f)

In [5]:
def get_best_validation_auprc(models_path):
    """
    Given the path to a set of runs, determines the run and epoch with the
    best validation auPRC for the first task.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}
    
    # Get the best auPRC
    best_auprc, best_run, best_epoch = 0, None, None
    for run_num in metrics.keys():
        val_auprcs = metrics[run_num]["val_corr_auprc"]["values"]
        for epoch_num, auprc_list in enumerate(val_auprcs):
            auprc = auprc_list[0]  # First one, arbitrarily
            if auprc > best_auprc:
                best_auprc, best_run, best_epoch = auprc, run_num, epoch_num + 1
    print("Best auPRC: %f" % best_auprc)
    print("Epoch %d in run %s" % (best_epoch, best_run))

In [8]:
def get_validation_auprcs(models_path):
    """
    Given the path to a set of runs, determines the run and epoch with the
    best validation auPRC for the first task.
    """
    # Get the metrics, ignoring empty or nonexistent metrics.json files
    metrics = {run_num : import_metrics_json(models_path, run_num) for run_num in os.listdir(models_path)}
    metrics = {key : val for key, val in metrics.items() if val}
    
    # Get the auPRCs
    auprcs = []
    for run_num in metrics.keys():
        val_auprcs = metrics[run_num]["val_corr_auprc"]["values"]
        best_auprc = 0
        for epoch_num, auprc_list in enumerate(val_auprcs):
            auprc = auprc_list[0]  # First one, arbitrarily
            best_auprc = max(best_auprc, auprc)
        auprcs.append(best_auprc)
    return auprcs

In [9]:
models_path = "/users/amtseng/att_priors/models/trained_models/SPI1_DREAM_priors/"
get_validation_auprcs(models_path)

[0.4899827897390704,
 0.476423933405104,
 0.4117989016584284,
 0.44645366982840595,
 0.49426116776511,
 0.45269926960865114,
 0.4614295314275299,
 0.41885080434858957,
 0.24951437545808186,
 0.4681138533039676,
 0.4827680207554059,
 0.41223663315775627,
 0.42430851940263514,
 0.24539632087738744,
 0.2942776378493398,
 0.47574228240336014]

In [10]:
models_path = "/users/amtseng/att_priors/models/trained_models/SPI1_DREAM/"
get_validation_auprcs(models_path)

[0.45219411889582534,
 0.46867854452851176,
 0.48039406056216316,
 0.48818340797364046,
 0.46070010727862243,
 0.4722176473315698,
 0.47995833825539963,
 0.4906607825076228,
 0.28592367196768825,
 0.4297766459964562,
 0.14994105008975595,
 0.46720758081444236,
 0.485427000019288,
 0.3266425474706105,
 0.47822219365714114,
 0.46526922276761107,
 0.4706664455964428,
 0.3526800168884162,
 0.3277461884033924,
 0.25045605686992184,
 0.47485940493217044,
 0.24791173792766985,
 0.43398303552647155,
 0.27285234637202693,
 0.4606782407423804,
 0.4793324035337665,
 0.301559999917848,
 0.46960798407766297,
 0.3517970108129185,
 0.4790455076721283,
 0.5046512905226174,
 0.4698410142258431,
 0.3503207088739436,
 0.4728151452278137,
 0.28233543453850535,
 0.47469914493456566,
 0.3483141562667044,
 0.471394776704998,
 0.5003524258810679,
 0.461808791786084,
 0.4794750628863745,
 0.4491154586039238,
 0.20515875906776684,
 0.4623344360344787,
 0.4789475390644602,
 0.4941814103732509,
 0.381162586939537